## éclat: system guidebook

### Preface

This guidebook is an integrated system design, theoretical reference, and operational runbook for `éclat` — an orchestration layer for conversational consumer lending. It is written for product leads, ML researchers, compliance officers, and engineering teams. Read sequentially for the full argument; use the Appendix for artifacts, schemas, and runnable examples.

---

### Table of contents (reading flow)

1. Executive summary  
2. Problem, scope, and objectives  
3. Design principles and operational constraints  
4. System architecture and runtime flow  
5. Formal models, derivations, and worked examples  
6. Master Agent: APIs, algorithms, and formal properties  
7. Worker Agents: contracts, algorithms, and safety  
8. Artifacts, schemas, and provenance  
9. Evaluation methodology and experiments  
10. Deployment, monitoring, and runbook  
11. Governance, compliance, and appendix

---

### 1. Executive summary

éclat models loan application journeys as temporally‑extended conversational processes rather than stateless form fills. A deterministic, observable Master Agent maintains a temporal state representation and orchestrates specialized Worker Agents (Sales, Verification, Underwriting, Sanction). The system trades opaque model complexity for reproducibility and auditability by using deterministic feature transforms, compact linear surrogates for scoring and attribution, signed append‑only artifacts, and conservative gating when uncertainty or manipulation risk is high.

Business objective: recover abandonment at critical friction points (verification, underwriting) while ensuring non‑manipulative, explainable automation that meets regulatory expectations.

---

### 2. Problem, scope, and objectives (clear framing)

Problem: digital lending funnels exhibit high abandonment (30–45% at verification/underwriting). Naïve automation may increase conversions but can introduce manipulation risk and regulatory exposure.

Scope: conversational/textual UI events (tokens, edits, pauses, clicks) + primary external adapters (credit bureau, KYC). Initial vertical: Sales → Verification → Underwriting.

Measurable objectives: reduce abandonment by 5 percentage points; preserve or increase NPS; maintain full artifact provenance for all automated decisions.

---

### 3. Design principles and operational constraints

Principles: interpretability first; conservative automation; auditability; lowest reasonable friction for users.

Constraints: explainability and nondiscrimination requirements, latency/scale constraints for production, privacy/consent limits on retention and signal collection.

Design consequence: prefer deterministic feature extractors and small, auditable surrogates for decisioning; only allow complex generative models behind strict filters and post‑hoc attribution.

---

### 4. System architecture and runtime flow (concise runtime narrative)

High‑level flow: Ingest → Feature Extractor → Master Agent (state graph + gating) → Worker Agents (candidate actions) → Safety Filter → Emit artifact & execute.

Runtime responsibilities: ingest attaches `trace_id` and provenance, extractor yields deterministic `f_t`, Master Agent updates graph and computes Readiness/Risk/Uncertainty, Worker Agents provide candidate actions/outputs, safety filter enforces policies and records signed artifacts.

Implementation note: persist artifacts to an append‑only store (Postgres JSONB/Timescale) with HMAC digests and commit tags linking to `policy_version` and code artifacts for reproducibility.

---

### 5. Formal models, derivations, and worked examples

This section presents the mathematical backbone used for guarantees, attribution, and policy mapping. Short proofs/sketches and a worked numeric example are included.

5.1 Temporal state graph: definitions and update rule

Let $V$ be a finite set of conversational states (greeting, clarification, verification_prompt, hesitation, productive_offer, abandon, sanction_preview, etc.).

At time $t$ the Master Agent maintains a directed graph $G_t=(V,E,W(t))$ where $W(t)$ assigns nonnegative weights $w_{ij}(t)$ to edges $(i\to j)$.

Signal update (exponential decay aggregator): incoming event signals $s_{ij}(\tau)$ (evidence for $i\to j$ at time $\tau$) update weights by a discounted sum:

$$
w_{ij}(t) = \sum_{\tau\le t} \alpha^{t-\tau} \, s_{ij}(\tau), \qquad 0<\alpha<1.
$$

Interpretation: $w_{ij}(t)$ measures discounted evidence for the transition $i\to j$ over recent history. This form supports (1) efficient streaming updates, (2) bounded memory via truncation, and (3) analytic bounds on expected hitting times under simple Markov approximations.

Worked example (numeric): suppose $\alpha=0.9$ and recent signals for $i\to j$ were $s = [1,0,2]$ at $\tau = t-2,t-1,t$ respectively; then

$$
w_{ij}(t)=0.9^{2}\cdot 1 + 0.9^{1}\cdot 0 + 0.9^{0}\cdot 2 = 0.81 + 0 + 2 = 2.81.
$$

5.2 Deterministic features and linear surrogate (scoring & exact attribution)

Let $\phi$ map event history $X_{\le t}$, persistent metadata $M$, and external signals $E$ to a deterministic feature vector $f_t \in \mathbb{R}^d$.

Use a linear surrogate

$$
S_t = w^\top f_t, \qquad p_t = \sigma(S_t),
$$

with $\sigma$ the logistic link. Attribution is exact: the contribution of feature $i$ is $w_i f_{t,i}$.

Fitting $w$: use constrained convex optimization (regularized logistic or linear objectives), optionally enforcing sign constraints or sparsity to align with policy.

5.3 Manipulation index and formal mitigation

Identify persuasion‑related features index set $P$ (urgency tokens, scarcity markers, loss framing counts, repeated prompts) and nonnegative severity weights $\beta$.

Define the manipulation index

$$
M_t = \sum_{i\in P} \beta_i \max\bigl(0,\, w_i f_{t,i}\bigr).
$$

Deterministic mitigation policy: if $M_t > M_{\mathrm{th}}$ and credit proxy $C_t < C_{\min}$ then reduce Sales urgency contribution by

$$
\Delta U = \lambda (M_t - M_{\mathrm{th}})
$$

and emit a `forced_reframe` event. This linear mitigation is reversible and testable by replaying historical traces.

5.4 Uncertainty and conservative gating

Estimate epistemic uncertainty $U_t$ using ensemble surrogates or prediction intervals on $S_t$. Compute a risk proxy $Q_t$ from external signals and anomaly detectors.

Decision rule (displayed as LaTeX):

$$
\text{Action}_t =
\begin{cases}
\text{escalate} & Q_t > q_0, \\
\text{pause/clarify} & Q_t \le q_0 \;\text{and}\; U_t > u_0, \\
\text{invoke worker} & Q_t \le q_0 \;\text{and}\; U_t \le u_0.
\end{cases}
$$

This discrete mapping ensures conservative automation and is amenable to formal verification: replaying the same input trace with fixed $\phi$, $\Theta$, and $\Pi$ yields identical outputs (reproducibility).

5.5 Reproducibility property (sketch)

Assume (i) deterministic $\phi$, (ii) fixed policy artifact versions $\Pi$, (iii) fixed numeric parameters $\Theta$, and (iv) deterministic seeding for any permitted generator components. Then the mapping

$$
(\mathcal{D}_t, T_t, Z_t) = F\bigl(X_{\le t}, M, E; \Theta, \Pi\bigr)
$$

is deterministic and replayable: identical inputs + identical policy/version produce identical signed artifacts $Z_t$.

---

### 6. Master Agent — API, algorithms, and properties

API primitives (deterministic)

- `decide(state_snapshot) -> {action, payload, reason_code}`
- `invoke(agent, payload)`
- `send(template_id, personalization)`
- `escalate(reason_code)`
- `record_artifact(Z)` — persists signed artifact.

Core algorithm (outline)

1. On event: update $f_t$ and $G_t$.  
2. Compute $S_t, p_t, M_t, R_t$ (readiness), $Q_t$ (risk), $U_t$ (uncertainty).  
3. Generate candidate actions with expected utility estimates using surrogate models.  
4. Apply safety filters (manipulation, risk, uncertainty). Block or modify candidates as per policy.  
5. Emit signed artifact $Z_t$ and execute allowed action.

Formal property: determinism under fixed $\Pi$ and deterministic $\phi$ implies audit replayability.

---

### 7. Worker Agents — contracts, algorithms, and safety

General contract: each Worker accepts typed JSON payloads and returns structured outputs plus attribution vectors.

7.1 Sales Agent — contract & safety

- Input: `{'state_snapshot', 'resonance_features'}`
- Output: `{'utterance_suggestion', 'offer_object', 'persuasion_attributions'}`
- Safety: Sales must compute manipulation attribution and set `manipulation_flag`; if flagged, Sales must use non‑urgent templates and log the event.

7.2 Verification Agent

- Input: `{'doc_images', 'id_fields'}`
- Output: `{'verified_flag', 'confidence', 'corrections'}`

Algorithms: deterministic prefilters (checksums, regex), phonetic matching, and embedding similarity (pgvector) with deterministic thresholds and deterministic fallback prompts.

7.3 Underwriting Agent

- Input: `{'features', 'external_scores'}`
- Output: `{'score', 'decision', 'shap_attributions', 'explanation_template'}`

Design: symbolic policy engine for primary rules plus surrogate ranking for borderline cases. Explanations derive from exact surrogate contributions.

7.4 Sanction Agent

- Input: `{'offer_terms', 'identity_status'}`
- Output: `{'doc_uri', 'template_commit', 'content_hash'}`

Design: deterministic template rendering with version tags and deterministic cryptographic signing of previews.

---

### 8. Artifacts, schemas, and provenance (operational requirements)

Minimal artifact schema (JSON):

```json
{
  "trace_id": "uuid",
  "lead_id": "L123",
  "timestamp": "2025-12-18T12:00:00Z",
  "schema_version": "1.0",
  "features": {"urgency_token_count": 2, "pause_ms": 450},
  "weights": {"w_urgency": 0.4},
  "score": 0.72,
  "decision": "invoke_sales",
  "reason_code": "R_HESITATION_01",
  "policy_version": "p-2025-12-01",
  "generator_version": "g-2025-12-10",
  "signature": "hmac:..."
}
```

Storage: append‑only JSONB with encryption at rest; index by `trace_id`, `lead_id`, and `policy_version` for replay; artifacts must be exported with redaction options for PII and with retained signatures for audit.

---

### 9. Evaluation methodology and experiments

9.1 Shadow mode and offline replay

Start by running the Master Agent in shadow mode: log decisions and compare to baseline; do not execute actions. Use replay to validate reproducibility and compute counterfactual KPIs.

9.2 A/B and phased rollout

Randomize at the trace level. Primary metrics: conversion, abandonment, approval‑to‑disbursal conversion, NPS. Use stratified sampling and bootstrap CIs; for proportions use z‑tests or permutation tests depending on sample size.

9.3 Safety metrics

Manipulation alert rate, false positive rate of persuasion detection, human override frequency, audit completeness. Maintain a labeled safety dataset and run periodic calibration.

---

### 10. Deployment, monitoring, and operational runbook

Phased deployment: offline → canary → ramp → production. Use feature flags and policy_version pinning.

Monitoring signals: feature drift (KL/JS divergence), decision entropy, escalation spikes, generator_version mismatches. Circuit breakers: automated rollback on defined thresholds.

Runbook highlights: how to replay a trace, how to fetch all artifacts for a `policy_version`, how to generate signed evidence for regulatory requests.

---

### 11. Governance, compliance, and appendix

Map artifact fields to compliance requirements (explain reasons for denial, retention windows, human oversight logs). Maintain governance docs tying `policy_version` to policy reviewers and approvers.

Appendix A: example artifacts — see `artifacts/` (I can inject a few curated JSON examples here if you want).  
Appendix B: selected bibliography (Lundberg & Lee 2017; Ribeiro et al. 2016; Doshi‑Velez & Kim 2017; Clark & Brennan 1991; operational responsible AI guides).

---